In [2]:
#gpt4omini, llama317B
generation_model = 'gpt4omini'

In [3]:
import os
from openai import OpenAI
from pydantic import BaseModel
import json

In [4]:
client = OpenAI(
    api_key = os.getenv('OPEN_AI_API')
)

In [5]:
class breakfast(BaseModel):
    name: str
    address: str

class morning_attraction(BaseModel):
    name: str
    address: str

class lunch(BaseModel):
    name: str
    address: str

class afternoon_attraction(BaseModel):
    name: str
    address: str

class dinner(BaseModel):
    name: str
    address: str

class night_attraction(BaseModel):
    name: str
    address: str

class accommodation(BaseModel):
    name: str
    address: str

class OneDay(BaseModel):
    days: str
    breakfast: breakfast
    morning_attractions: list[morning_attraction]
    lunch: lunch
    afternoon_attractions: list[afternoon_attraction]
    dinner: dinner
    night_attractions: list[night_attraction]
    accommodation: accommodation

class WholePlan(BaseModel):
    itinerary: list[OneDay]

In [6]:
with open('Utils/Plan_extraction_task1.txt', 'r') as file:
    system_prompt = file.read()

In [7]:
system_prompt

'Extract the travel itinerary and parse the businesses\' information into the JSON format as below. Be faithful and concise. Correctly document the right number of the attractions. Only write down the name and address of the businesses. If certain recommendations (like meals or accommodations) are not provided, replace the information with "-" for name and address. If recommendations for a session of attraction is not provided, replace the information as an empty array. \n\n----- Example Starts -----\n{\n    "itinerary":[\n        {   \n            "days": "x",\n            "breakfast": {\n                "name": "xxx",\n                "address": "xxx"\n            },\n            "morning_attractions": [\n                {\n                    "name": "xxx",\n                    "address": "xxx"\n                },\n                {\n                    "name": "xxx",\n                    "address": "xxx"\n                }\n            ],\n            "lunch": {\n                "n

In [9]:
def extract_plan(user_prompt, system_prompt):
    chat_completion = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        temperature=0,
        messages=[
            {"role": "system","content": system_prompt,},
            {"role": "user","content": user_prompt,}
        ],
        response_format=WholePlan
    )

    return json.loads(chat_completion.choices[0].message.parsed.json())

In [10]:
def extract_gpt4omini_plan():
    for filename in os.listdir('Outputs/plans/gpt4omini'):
        index = filename[:-4][5:]
        with open(f'Outputs/plans/gpt4omini/{filename}', 'r') as file:
            user_prompt = file.read()
            plan_eval = extract_plan(system_prompt, user_prompt)
            with open(f'Outputs/evals/gpt4omini/Plan_Eval_{index}.json', 'w') as f:
                json.dump(plan_eval, f, indent=4)

    #post process of the evaluation of each plan

    #carry on the hotel if not mentioned specifically
    for filename in os.listdir('Outputs/evals/gpt4omini'):
        with open(f'Outputs/evals/gpt4omini/{filename}', 'r') as file:
            plan_eval = json.load(file)
            
            hotel_list = []
            hotel_carriedOn = []
            current_hotel = {"name":"-", "address":"-"}

            for day in plan_eval['itinerary']:
                hotel_list.append(day['accommodation'])
                
            for item in hotel_list:
                if item != {"name":"-", "address":"-"}:
                    current_hotel = item
                hotel_carriedOn.append(current_hotel)
            
            for i in range(len(plan_eval['itinerary'])):
                plan_eval['itinerary'][i]['accommodation'] = hotel_carriedOn[i]
        
        with open(f'Outputs/evals/gpt4omini/{filename}', 'w') as f:
            json.dump(plan_eval, f, indent=4)

In [9]:
def extract_llama317B_plan():
    for filename in os.listdir('Outputs/plans/Llama317B'):
        index = filename[:-4][5:]
        with open(f'Outputs/plans/Llama317B/{filename}', 'r') as file:
            user_prompt = file.read()
            plan_eval = extract_plan(system_prompt, user_prompt)
            with open(f'Outputs/evals/Llama317B/Plan_Eval_{index}.json', 'w') as f:
                json.dump(plan_eval, f, indent=4)

    #post process of the evaluation of each plan

    #carry on the hotel if not mentioned specifically
    for filename in os.listdir('Outputs/evals/Llama317B/'):
        with open(f'Outputs/evals/Llama317B/{filename}', 'r') as file:
            plan_eval = json.load(file)
            
            hotel_list = []
            hotel_carriedOn = []
            current_hotel = {"name":"-", "address":"-"}

            for day in plan_eval['itinerary']:
                hotel_list.append(day['accommodation'])
                
            for item in hotel_list:
                if item != {"name":"-", "address":"-"}:
                    current_hotel = item
                hotel_carriedOn.append(current_hotel)
            
            for i in range(len(plan_eval['itinerary'])):
                plan_eval['itinerary'][i]['accommodation'] = hotel_carriedOn[i]
        
        with open(f'Outputs/evals/Llama317B/{filename}', 'w') as f:
            json.dump(plan_eval, f, indent=4)

In [11]:
if generation_model == 'llama317B':
    extract_llama317B_plan()
if generation_model == 'gpt4omini':
    extract_gpt4omini_plan()